In [26]:
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import re

In [27]:
df = pd.read_csv("../output/dfcompaniesSF.csv")
df.head()

,Unnamed: 0,name,category_code,number_of_employees,founded_year,city,office,state,total_money_raised ($k),money_state
0,6,Digg,news,60.0,2004.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...",success,45000.0,success
1,8,Scribd,news,50.0,2007.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.404052,...",success,25800.0,success
2,14,StumbleUpon,web,NaN,2002.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,...",success,18500.0,success
3,22,Twitter,social,1300.0,2006.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.4169244...",success,1160000.0,success
4,24,Powerset,search,60.0,2006.0,San Francisco,"{'type': 'Point', 'coordinates': [-122.395289,...",success,22500.0,success


In [28]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [32]:
new_companies = pd.DataFrame(df["office"][df["founded_year"]>2003])

In [33]:
new_companies.to_json("../output/new_companies.json",orient="records")

In [29]:
#Voy a empezar con el requisito de no tener en 2 km compañías de más de 10 años de antigüedad
old_companies = pd.DataFrame(df["office"][df["founded_year"]<=2003])
old_companies.head()

,office
2,"{'type': 'Point', 'coordinates': [-122.419204,..."
5,"{'type': 'Point', 'coordinates': [-122.393041,..."
22,"{'type': 'Point', 'coordinates': [-122.397977,..."
28,"{'type': 'Point', 'coordinates': [-122.389757,..."
30,"{'type': 'Point', 'coordinates': [-122.40184, ..."


In [30]:
def getCoordinates(row):
    value = row.office
    if type(value)==str:
        coord = value.split("[")[1].split(",")
        long = float(coord[0])
        lat = float(coord[1].split("]")[0])
        return (long,lat)
    
old_companies = old_companies.apply(getCoordinates,axis=1,result_type="expand")
old_companies.columns=["longitude","latitude"]


In [31]:
old_companies.head()

,longitude,latitude
2,-122.419204,37.775196
5,-122.393041,37.779558
22,-122.397977,37.778076
28,-122.389757,37.789803
30,-122.401840,37.779026


In [52]:
old_companies.isnull().sum()
nulls = old_companies.isnull().sum(axis=1)[old_companies.isnull().sum(axis=1)==2].index
old_companies = old_companies.drop(index=nulls)
old_companies

,longitude,latitude
2,-122.419204,37.775196
5,-122.393041,37.779558
22,-122.397977,37.778076
28,-122.389757,37.789803
30,-122.401840,37.779026
...,...,...
887,-122.402759,37.787646
890,-122.402021,37.764169
894,-122.404392,37.786905
895,-122.400655,37.789318


In [54]:
print(list(old_companies.index))

[2, 5, 22, 28, 30, 39, 40, 42, 45, 51, 54, 56, 68, 69, 72, 80, 85, 94, 110, 129, 131, 137, 142, 143, 146, 169, 173, 180, 182, 183, 185, 190, 195, 197, 212, 224, 231, 244, 257, 263, 266, 270, 275, 282, 287, 288, 289, 291, 299, 311, 315, 316, 324, 334, 342, 348, 355, 358, 364, 366, 372, 374, 384, 387, 390, 391, 397, 400, 417, 418, 419, 421, 427, 430, 442, 456, 462, 472, 476, 495, 503, 504, 510, 516, 521, 525, 544, 552, 554, 559, 565, 582, 584, 591, 594, 598, 619, 622, 624, 632, 642, 646, 648, 654, 656, 659, 665, 672, 675, 683, 687, 697, 700, 705, 710, 717, 722, 728, 730, 731, 732, 735, 737, 761, 770, 771, 807, 811, 816, 817, 820, 821, 823, 828, 839, 848, 867, 869, 880, 881, 886, 887, 890, 894, 895, 904]


In [78]:
companies_FAR = []
for i in range(len(old_companies)):
    
    query = {'office': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [round(old_companies["longitude"].iloc[i],6),round(old_companies["latitude"].iloc[i],6)],
   '$minDistance': 2000}}}}

    companies_FAR.append(db.companies_SF.find(query, {"name":1,"office":1,"_id":0}))

In [80]:
lista_alloffcies = []
for cur in companies_FAR:
    lista_alloffcies.append(list(cur))

In [81]:
lista_offices = [comp for lista in lista_alloffcies for comp in lista]

In [82]:
for i in range(len(lista_offices)):
    a=0
    if lista_offices[a] == lista_offices[i+1]:
        print(lista_offices[i])
        a=0
    else:
        a+1=

[{'name': 'Technorati',
  'office': {'type': 'Point', 'coordinates': [-122.393041, 37.779558]}},
 {'name': 'Pixsy Image Search',
  'office': {'type': 'Point', 'coordinates': [-122.393041, 37.779558]}},
 {'name': 'IntoSpace',
  'office': {'type': 'Point', 'coordinates': [-122.393041, 37.779558]}},
 {'name': 'PowerReviews',
  'office': {'type': 'Point', 'coordinates': [-122.393041, 37.779558]}},
 {'name': 'GroupSwim',
  'office': {'type': 'Point', 'coordinates': [-122.393041, 37.779558]}},
 {'name': 'ArcTouch',
  'office': {'type': 'Point', 'coordinates': [-122.392343, 37.779868]}},
 {'name': 'SpongeFish',
  'office': {'type': 'Point', 'coordinates': [-122.3943096, 37.7800788]}},
 {'name': 'CastTV',
  'office': {'type': 'Point', 'coordinates': [-122.393913, 37.780716]}},
 {'name': 'imeem',
  'office': {'type': 'Point', 'coordinates': [-122.391236, 37.779624]}},
 {'name': 'Smule',
  'office': {'type': 'Point', 'coordinates': [-122.391236, 37.779624]}},
 {'name': 'Fotomoto',
  'office': {'

In [74]:
print(lista_alloffcies)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [65]:
companiesFAR = pd.DataFrame(lista_companies)

In [66]:
companiesFAR

,name,office
0,Codegear,"{'type': 'Point', 'coordinates': [-122.398062,..."
1,Focus,"{'type': 'Point', 'coordinates': [-122.398062,..."
2,Embarcadero Technologies,"{'type': 'Point', 'coordinates': [-122.398062,..."
3,Piczo,"{'type': 'Point', 'coordinates': [-122.398479,..."
4,CodeGlide,"{'type': 'Point', 'coordinates': [-122.3979712..."
...,...,...
616,Viagogo,"{'type': 'Point', 'coordinates': [-0.126236, 5..."
617,Unity Technologies,"{'type': 'Point', 'coordinates': [12.530107, 5..."
618,Zendesk,"{'type': 'Point', 'coordinates': [12.568116, 5..."
619,Fortumo,"{'type': 'Point', 'coordinates': [24.8021233, ..."


In [ ]:
# Voy a limpiar la base de datos companies
companiesSF = db.companies_SF.find({},{"name":1,"offices":1,"total_money_raised":1,"number_of_employees":1,
                                   "founded_year":1,"category_code":1})